In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import ipywidgets as widgets
from ipywidgets import Layout

def nextpow2(i):
    n = 1
    while n < i: n *= 2
    return n

def pwelch(x,Fs):                    
    Ts=1/Fs                    
    L=np.size(x)+1          
    T=L*Ts                     
    N = 2^nextpow2(L)
    Fo=Fs/N                   
    f=np.arange(0,N)*Fo       
        
    window_size = nextpow2(np.size(x)/8)
    if (window_size<256):
        window_size=256
    windows = np.size(x)//(window_size//2)-1
    indexer = np.arange(window_size)[None, :] + (window_size//2)*np.arange(windows)[:, None]
    windowed_x = x[indexer]

    avg_pwr=0
    for window in windowed_x:
        window = window * np.hanning(np.size(window))
        L=np.size(window)+1                 
        T=L*Ts                     
        N = 2^nextpow2(L)
        Fo=Fs/N                   
        f=np.arange(0,N)*Fo
        window_fft=np.fft.fft(window,N)
        power=np.multiply(window_fft,np.conj(window_fft))/N/L
        avg_pwr=avg_pwr+power
    avg_pwr=avg_pwr/windows


    return f[np.arange(0,N//2)], avg_pwr[np.arange(0,N//2)]

# Function to update plots based on slider value
def update_plots(sampling_frequency):
    signal_length = 1000
    sample_period = 1 / sampling_frequency  # Update sampling period
    time_vector = np.arange(0, signal_length) * sample_period  # Update time vector
    
    # Create signal with updated sampling frequency
    signal_data = np.sin(2 * np.pi * 30 * time_vector) + 0.8 * np.sin(2 * np.pi * 80 * (time_vector - 2)) + np.sin(2 * np.pi * 60 * time_vector)
    
    # Compute custom pwelch
    custom_frequencies, custom_power = pwelch(signal_data, sampling_frequency)
    
    # Compute signal.welch
    welch_frequencies, welch_power = signal.welch(signal_data, fs=sampling_frequency)
    
    # Compute FFT
    L = len(signal_data)  # Length of signal
    N = 1 * L  # Length of Fourier Transform
    Fo = sampling_frequency / N  # Frequency resolution
    Fx = np.fft.fft(signal_data, N)  # DFT of the signal
    freq = np.arange(0, N) * Fo  # Frequency vector
    
    # Compute periodogram
    power = ((Fx * np.conj(Fx)) / (sampling_frequency * L)).real  # Calculate spectral density

    # Plot the results
    fig, axs = plt.subplots(4, 1, figsize=(15, 20))
    
	# Plot FFT
    axs[0].plot(freq, np.abs(Fx))
    axs[0].set(xlabel='Frequency (Hz)', ylabel='Magnitude', title='FFT of the Signal')
    axs[0].grid()

	# Plot periodogram
    axs[1].plot(freq, power)
    axs[1].set(xlabel='Frequency (Hz)', ylabel='Power', title='Periodogram')
    axs[1].grid()

    # Plot custom pwelch
    axs[2].plot(custom_frequencies, custom_power)
    axs[2].set(xlabel='Frequency (Hz)', ylabel='Power', title='Periodogram (Custom pwelch)')
    axs[2].grid()
    
    # Plot signal.welch
    axs[3].plot(welch_frequencies, welch_power)
    axs[3].set(xlabel='Frequency (Hz)', ylabel='Power', title='Periodogram (signal.welch)')
    axs[3].grid()

    

    plt.tight_layout()
    plt.show()

# Create slider for sampling frequency
sampling_frequency_slider = widgets.IntSlider(
    value=500,
    min=100,
    max=2000, 
    step=100,
    description='Sampling Frequency (Fs):',
    layout=Layout(width='auto', flex='1 1 auto'),
    style={'description_width': 'initial'}, 
    continuous_update=False
)

# Display the sliders and output
vbox_layout = Layout(display='flex', flex_flow='column', align_items='center')

# Group the frequency inputs together
inputs_box = widgets.HBox([sampling_frequency_slider], layout=Layout(flex='1 1 auto', width='auto'))

ui = widgets.HBox([inputs_box], layout=Layout(display='flex', justify_content='center', width='100%', align_items='center'))

output = widgets.interactive_output(update_plots, {'sampling_frequency': sampling_frequency_slider})

# Display UI and plot
display(ui, output)


Output()